In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
import os
import cv2
import hashlib
from PIL import Image
print("Pillow is installed!")

Pillow is installed!


In [7]:
train_dir="C:/Users/Lenovo/Desktop/Project-1/train"
test_dir="C:/Users/Lenovo/Desktop/Project-1/test"

data cleaning

In [3]:
# img_size=(48,48)

# def get_hash(image_path):
#   with open(image_path,"rb") as f:
#     return hashlib.md5(f.read()).hexdigest()

# hashes=set()

# for emotion in os.listdir(train_dir):
#   emotion_path=os.path.join(train_dir,emotion)

#   for filename in os.listdir(emotion_path):
#     img_path=os.path.join(emotion_path,filename)

#     try:
#       img=cv2.imread(img_path)
#       if img is None:
#         print(f"Corrupt image found and removed: {img_path}")
#         os.remove(img_path)
#         continue

#       img_hash=get_hash(img_path)
#       if img_hash in hashes:
#         print(f"Duplicate image found and removed: {img_path}")
#         os.remove(img_path)
#         continue

#       else:
#         hashes.add(img_hash)

#       img=Image.open(img_path).convert("L")
#       img=img.resize(img_size)
#       img.save(img_path)

#     except Exception as e:
#       print(f"Error processing {img_path}: {e}")
#       os.remove(img_path)


Data Augmentation

In [5]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    horizontal_flip=True
)

test_datagen=ImageDataGenerator(rescale=1./255)

In [8]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(48,48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 27473 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


CNN Model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    # Convolutional layers
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    # Flatten and fully connected layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(7, activation='softmax')  # 7 classes for emotions
])

# Show model summary
model.summary()

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 2, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 523,015 (2.00 MB)

 Trainable params: 523,015 (2.00 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from keras.optimizers import SGD

model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),  # Using SGD optimizer with momentum
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=200,
    verbose=1
)

Epoch 1/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 107s 250ms/step - accuracy: 0.5893 - loss: 1.0805 - val_accuracy: 0.6272 - val_loss: 1.0179
Epoch 2/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 77s 179ms/step - accuracy: 0.5870 - loss: 1.0821 - val_accuracy: 0.6282 - val_loss: 1.0017
Epoch 3/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 67s 157ms/step - accuracy: 0.5930 - loss: 1.0728 - val_accuracy: 0.6173 - val_loss: 1.0308
Epoch 4/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 60s 139ms/step - accuracy: 0.5952 - loss: 1.0797 - val_accuracy: 0.6232 - val_loss: 1.0052
Epoch 5/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 59s 136ms/step - accuracy: 0.5936 - loss: 1.0821 - val_accuracy: 0.6128 - val_loss: 1.0294
Epoch 6/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 60s 139ms/step - accuracy: 0.5868 - loss: 1.0924 - val_accuracy: 0.6234 - val_loss: 1.0241
Epoch 7/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 64s 148ms/step - accuracy: 0.5871 - loss: 1.0905 - val_accuracy: 0.6212 - val_loss: 1.0276
Epoch 8/200
430/430 ━━━━━━━━━━━━━━━━━━━━ 65s 150ms/step - accuracy: 0.5894 

In [14]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


225/225 ━━━━━━━━━━━━━━━━━━━━ 29s 117ms/step - accuracy: 0.6414 - loss: 1.0167
Test Accuracy: 0.64


In [14]:
# Save the entire model
model.save('Model2.h5')  # Saves model in .h5 format

In [1]:
from keras.models import load_model

# Load the model
model = load_model('Model2.h5')

# Now, you can use `model.predict()` for inference

In [ ]:
# import numpy as np
# from tensorflow.keras.preprocessing import image # type: ignore

# # Load an image for testing
# img_path = "C:/Users/Lenovo/Desktop/Project-1/test/disgust/PrivateTest_3881740.jpg"
# img = image.load_img(img_path, target_size=(48, 48), color_mode="grayscale")
# img_array = image.img_to_array(img) / 255.0
# img_array = np.expand_dims(img_array, axis=0)  # Make it batch format

# # Predict emotion
# predictions = model.predict(img_array)
# predicted_class = np.argmax(predictions)

# # Emotion labels
# emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
# print("Predicted Emotion:", emotion_labels[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Predicted Emotion: Angry


In [2]:
import numpy as np
import cv2
# Emotion labels (Modify as per your dataset)
emotions = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Initialize webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam

# Load face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract face ROI
        face = gray[y:y+h, x:x+w]

        # Resize to match model input size (48x48)
        face = cv2.resize(face, (48, 48))

        # Normalize & reshape
        face = np.expand_dims(face, axis=0)  # Add batch dimension
        face = np.expand_dims(face, axis=-1)  # Add channel dimension (grayscale)
        face = face / 255.0  # Normalize

        # Predict emotion
        prediction = model.predict(face)
        emotion_label = np.argmax(prediction)
        emotion_text = emotions[emotion_label]

        # Draw a rectangle around face & put emotion label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Show frame
    cv2.imshow("Live Emotion Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━